In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from pathlib import Path
from pandas.tseries.offsets import MonthEnd
from pmp_functions_v5 import (
    run_perf_summary_benchmark_vs_strategy,
    run_cc_strategy,
) 

In [8]:
## FUNDAMENTAL STRATEGY 
## --------------------
## This script implements a long/short fundamental strategy across regional equity indices.

# =============================================================================
# 0. CONFIGURATION
# =============================================================================
DATA_DIR = Path(r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data")

FUND_FILE      = DATA_DIR / "Fundamental_Indicators.xlsx"
FX_FILE        = DATA_DIR / "FX.xlsx"  
PRICE_FILE     = DATA_DIR / "Futures.xlsx" 

# --- UPDATED FILES ---
RF_FILE        = DATA_DIR / "RF.csv"               # Changed to CSV
BENCHMARK_FILE = DATA_DIR / "Benchmarks.xlsx"       # Added Benchmark file

# Volatility target for the whole strategy (annualized)
TARGET_VOL_ANN = 0.10

# Transaction cost per unit of *turnover* TCOST_PER_UNIT = 0.000

SIGNAL_CONFIG = [
    {"ind": "ADJ_PE",              "lag": 6, "sign": -1}, 
    {"ind": "PX_TO_BOOK_RATIO",    "lag": 6, "sign": -1}, 
    {"ind": "IS_EPS",              "lag": 6, "sign":  1}, 
    {"ind": "PX_TO_SALES_RATIO",   "lag": 6, "sign": -1}, 
    {"ind": "RETURN_ON_ASSET",     "lag": 6, "sign":  1}, 
]

In [9]:
# =============================================================================
# 1. DATA LOADING FUNCTIONS
# =============================================================================

def load_fundamentals_dict(path: Path) -> dict:
    """
    Load panel of fundamental indicators and return it as a dict of DataFrames.
    """
    df = pd.read_excel(path)
    
    # Ensure Date is datetime and aligned to month end
    df["Date"] = pd.to_datetime(df["Date"]) + MonthEnd(0)
    
    # Convert wide columns to long format
    long = df.melt(id_vars="Date", var_name="raw_col", value_name="value").dropna(subset=["value"])
    
    # Split raw_col into Region & Indicator
    long[["Region", "Indicator"]] = long["raw_col"].str.split("_", n=1, expand=True)
    
    # Pivot into a multi-index columns DataFrame
    full_panel = long.pivot_table(
        index="Date",
        columns=["Indicator", "Region"],
        values="value",
        aggfunc="mean"
    )
    
    # Split into dict by Indicator
    fund_dict = {}
    unique_inds = full_panel.columns.get_level_values(0).unique()
    for ind in unique_inds:
        fund_dict[ind] = full_panel[ind].sort_index()
        
    return fund_dict


def load_prices_futures_usd(path: Path) -> pd.DataFrame:
    """
    Load equity futures prices (already in USD) and compute monthly returns.
    """
    df = pd.read_excel(path)
    df["Date"] = pd.to_datetime(df["Date"])
    
    if "UK" in df.columns:
        df = df.rename(columns={"UK": "GB"})

    df = df.set_index("Date").sort_index()
    
    # Monthly returns in USD
    ret_panel = df.pct_change()
    
    # Align index to month end
    ret_panel.index = ret_panel.index + MonthEnd(0)
    
    # Handle duplicates
    ret_panel = ret_panel.groupby(level=0).last()
    
    return ret_panel


def load_rf_csv(path: Path) -> pd.Series:
    """
    Load risk-free rate from CSV. 
    Format: Date, RF (returns)
    """
    df = pd.read_csv(path)
    
    # Parse dates and align to MonthEnd
    df["Date"] = pd.to_datetime(df["Date"]) + MonthEnd(0)
    
    df = df.set_index("Date").sort_index()
    
    # Return the RF column
    return df["RF"]


def load_benchmark(path: Path) -> pd.Series:
    """
    Load Benchmark Excel, select MSCI World, compute returns.
    """
    df = pd.read_excel(path)
    
    # Parse dates and align to MonthEnd
    df["Date"] = pd.to_datetime(df["Date"]) + MonthEnd(0)
    df = df.set_index("Date").sort_index()
    
    # Ensure we select MSCI World and calculate returns
    # Assuming column name is strictly "MSCI World"
    prices = df["MSCI World"]
    
    # Calculate returns
    returns = prices.pct_change()
    
    return returns

In [ ]:
# =============================================================================
# 2. CALCULATION LOGIC (SIGNALS)
# =============================================================================
def compute_signals(fund_dict, returns_df, config):
    """
    Construct the composite fundamental signal and estimate rolling volatility.
    """
    # 1) Estimate rolling vol per region
    rolling_std = (
        returns_df
        .rolling(window=36, min_periods=36)
        .std()
        .replace(0, np.nan)
    )
    
    # 2) master_z will accumulate z-scores
    master_z = pd.DataFrame(0.0, index=returns_df.index, columns=returns_df.columns)
    valid_count = 0 
    
    # 3) Loop over each configured indicator
    for cfg in config:
        ind  = cfg["ind"]
        lag  = cfg["lag"]
        sign = cfg["sign"] 
        
        if ind in fund_dict:
            panel = fund_dict[ind]   

            mom = panel.diff(lag) * sign

            mu    = mom.mean(axis=1)
            sigma = mom.std(axis=1)

            z = mom.sub(mu, axis=0).div(sigma, axis=0).fillna(0)
            master_z = master_z.add(z, fill_value=0)

            valid_count += 1
            
    final_signal = master_z / valid_count if valid_count > 0 else master_z

    return rolling_std, final_signal


In [11]:
# =============================================================================
# 3. PERFORMANCE SUMMARY (Helper)
# =============================================================================
# Note: summarizePerformance is a helper. The main summary is likely 
# run_perf_summary_benchmark_vs_strategy which is external or imported.

def summarizePerformance(xsReturns, Rf, factorXsReturns, annualizationFactor=12, strategyNames=None):
    xsReturns        = np.array(xsReturns)
    factorXsReturns = np.array(factorXsReturns).flatten()
    
    metrics = []
    
    for i in range(xsReturns.shape[1]):
        r_xs = xsReturns[:, i] 
        
        mu = np.mean(r_xs) * annualizationFactor                   
        sig = np.std(r_xs, ddof=1) * np.sqrt(annualizationFactor)   
        sharpe = mu / sig if sig > 0 else 0
        
        if np.std(factorXsReturns) == 0 or np.std(r_xs) == 0:
            beta, alpha, corr = 0.0, 0.0, 0.0
        else:
            slope, intercept, r_value, _, _ = stats.linregress(factorXsReturns, r_xs)
            beta  = slope
            alpha = intercept * annualizationFactor 
            corr  = r_value
            
        metrics.append({
            "Ann. Return (XS)": mu,
            "Ann. Volatility":  sig,
            "Sharpe Ratio":     sharpe,
            "Beta":             beta,
            "Ann. Alpha":       alpha,
            "Correlation":      corr
        })
        
    df = pd.DataFrame(
        metrics,
        index=strategyNames if strategyNames else [f"Strat {i}" for i in range(len(metrics))]
    )
    return df.T

In [12]:
# =============================================================================
# 4. MAIN EXECUTION
# =============================================================================

# A. Load Data
print("Loading Data...")
fund_dict         = load_fundamentals_dict(FUND_FILE)
ret_unhedged_usd  = load_prices_futures_usd(PRICE_FILE)

# --- UPDATED LOADING CALLS ---
rf_series         = load_rf_csv(RF_FILE)
benchmark_returns = load_benchmark(BENCHMARK_FILE)

# B. Calculate Signals
vol_usd, signal = compute_signals(fund_dict, ret_unhedged_usd, SIGNAL_CONFIG)

# C. Calculate Weights
vol_target_monthly = TARGET_VOL_ANN / np.sqrt(12)
risk_scaler = vol_target_monthly / vol_usd
raw_weights = signal * risk_scaler

# D. Prepare Data for Backtest Engine
final_weights = raw_weights.reindex(ret_unhedged_usd.index).fillna(0)

print("Running Backtest (Unhedged Futures)...")

perf_df = run_cc_strategy(
    weights      = final_weights,
    returns      = ret_unhedged_usd.dropna(how='all'), 
    rf           = rf_series,
    t_cost       = 0,
    benchmark    = benchmark_returns,   # We run it with 'equal' initially, then overwrite below
    long_short   = True,      
    beta_neutral = False     
)

# --- CRITICAL ADJUSTMENT: OVERWRITE BENCHMARK ---
# The backtest creates a 'ret_bm' column (likely Equal Weight). 
# We overwrite it with MSCI World for the summary.

# 1. Align MSCI World returns to the backtest dates
aligned_msci = benchmark_returns.reindex(perf_df.index).fillna(0)

# 2. Inject into results DataFrame
perf_df["ret_bm"] = aligned_msci

# E. Plotting & Summary
perf_df = perf_df[perf_df["turnover"].cumsum() > 0]

cum_strat = (1 + perf_df["ret_net"]).cumprod()

# Calculate Excess Returns for Benchmark (MSCI World - RF) before plotting
bench_excess = perf_df["ret_bm"] - perf_df["ret_rf"]
cum_bench = (1 + bench_excess).cumprod()

print("\n--- PERFORMANCE SUMMARY (VS MSCI WORLD) ---")

# Now run the summary function. It will pick up the modified 'ret_bm' from perf_df
# NOTE: Assumes run_perf_summary_benchmark_vs_strategy is defined in your environment
summary_table = run_perf_summary_benchmark_vs_strategy(
    results=perf_df,
    alreadyXs=True, 
    annualizationFactor=12,
    strategyNames=["MSCI World", "Fund L/S (Unhedged)"] # Updated Label
)
print(summary_table)

# --- DETAILED MONTHLY RETURNS & WEIGHTS ---
print("\n--- DETAILED MONTHLY RETURNS & WEIGHTS ---")
cols_to_show = ["ret_net", "ret_gross", "ret_rf", "ret_bm"] + [
    c for c in perf_df.columns if c.startswith("w_")
]

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(perf_df[cols_to_show])

# --------------------------------------------------
# Plot performance & weights
plt.figure(figsize=(12, 10))

# Plot 1: Cumulative wealth 
plt.subplot(2, 1, 1)
plt.plot(cum_strat.index, cum_strat, label="Fund L/S (Unhedged, Net)", color='blue', linewidth=2)
plt.plot(cum_bench.index, cum_bench, label="MSCI World (Excess Return)", color='gray', linestyle="--") # Updated Label
plt.title("Strategy Performance vs MSCI World (Excess)")
plt.ylabel("Cumulative Wealth")
plt.legend()
plt.grid(True, alpha=0.3)



plt.tight_layout()
plt.show()

Loading Data...


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Fundamental_Indicators.xlsx'